In [2]:
import pandas as pd
import json
import os.path

In [27]:
from medcat.cat import CAT
from medcat.utils.vocab import Vocab
from medcat.cdb import CDB
from medcat.meta_cat import MetaCAT

In [5]:
import pandas as pd

import numpy as np
import json
from datetime import datetime
import os.path

### Notes

In [3]:
notes = pd.read_csv("data/mimic_notes.csv")

/users/k1810895/.conda/envs/medcat/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (4,5,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# notes.head()

In [6]:
notes[notes['row_id'].isna()==True]

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text,gender,dob,dod,dod_hosp,dod_ssn,expire_flag,etl_date


In [7]:
notes.shape

(2083180, 18)

## Medcat annotations

In [8]:
max(range(10))

9

In [35]:
# medcat
batch = 1003
# ds = set()
for i in range(1002, 1003):
    try:
        d = json.load(open('data/medcat/batch' + str(i) +'.json',"r"))
    except Exception as e:
        print(i)
        continue
    for did in d.keys():
        ds.add(int(did))

In [18]:
list(ds)[:3]

[1, 2, 3]

In [36]:
len(ds)

2083180

In [37]:
len(ds)

2083180

In [38]:
1 in set(notes['row_id'])

True

In [39]:
miss_rows = (set(notes['row_id']) - (ds))

In [43]:
len(miss_rows)

0

In [41]:
list(miss_rows)[:3]

[]

In [42]:
1006 == 1006

True

In [14]:
notes['row_id'].tolist()[:3]

[673887, 673893, 673895]

In [15]:
list(miss_rows)[:3]

[1048581, 1048596, 524309]

In [16]:
# list(miss_rows)

In [17]:
notes.shape

(2083180, 18)

In [18]:
2083180 - 1852549

230631

In [19]:
230631/2083180

0.11071102833168521

#### A large number of documents missing. One possible reason is: the annotations cannot be saved if multiple nodes are used. 

In [28]:
cdb = CDB()
# cdb.load_dict("/home/ubuntu/Tao/KER/medcat_models/0.2.6.2 - snomed_us_ext_names_umls_clean_primary_1M.dat")
cdb.load_dict("/users/k1810895/data/KER/medcat_models/umls_base_wlink_clean_name_400k_mimic.dat")
vocab = Vocab()
vocab.load_dict(path='/users/k1810895/data/KER/medcat_models/base_vocabulary.dat')


mc_negated = MetaCAT(save_dir="/users/k1810895/data/KER/medcat_models/mc_negated/")
mc_negated.load()
mc_skip = MetaCAT(save_dir="/users/k1810895/data/KER/medcat_models/mc_skip/")
mc_skip.load()


cat = CAT(cdb, vocab=vocab, meta_cats=[mc_negated, mc_skip])
cat.train = False
cat.spacy_cat.MIN_ACC = 0.35
cat.spacy_cat.MIN_CONCEPT_LENGTH = 2

In [29]:
notes['row_id'].head()

0    673887
1    673893
2    673895
3    674005
4    672943
Name: row_id, dtype: int64

In [30]:
df = notes[notes['row_id'].isin(miss_rows) == True]

In [31]:
df.shape

(600, 18)

In [32]:
batch_size = 1000 # Use 10k if you have a lot of documents
batch = []
cnt = 0
docs = {}

for name, text in df[['row_id', 'text']].values:
    text = str(text)
    name = str(name)
#     print(datetime.now(), ' document ', name)
    cnt += 1
    if len(text) > 0 and name not in docs:
        # This will screwup positions of entites if doc>100k
        npart = int(np.ceil(len(text) / 100000))
        for i in range(npart):
            batch.append((name, text[i*100000:(i+1)*100000]))


    if len(batch) >= batch_size or (cnt == len(df) and len(batch) > 0):
        res = cat.multi_processing(batch, nproc=10, batch_size=batch_size // 100) # Use the correct number of processors

        for name, doc in res:
            if name not in docs:
                docs[name] = []

            docs[name].extend(doc['entities'])
#             print(name)
        batch = []

In [33]:
json.dump(docs, open('data/medcat/batch' + str(1002) +'.json', 'w'))  

In [3]:
d = json.load(open('data/medcat/batch1001.json',"r"))

In [14]:
from itertools import islice

def chunks(data, SIZE=10000):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}

In [15]:
len(d)

230031

In [16]:
230031/100

2300.31

In [17]:
i = 1
for item in chunks(d, 2300):
    json.dump(item, open('data/medcat/batch2_' + str(i) +'.json', 'w')) 
    i += 1